# Part 1: Matching Datasets

## Project intro / part 1 intro

This project focusses on altitude vs population... *

The primary dataset used is... which does not contain elevation, the elevation is extracted from a secondary dataset... . The two datasets to not share the same unique identifier for each town - it is necessary to join based on the town name and country. This doesn't provide a unique match for each town, so the population and latitude/longitude are used to identify matches. Part 1 of this project aims to merge the 2 datasets.

## Data Sources
    - description
        - context (i.e. date)
        - important columns



## Matching Process
    - mexican town of (?)
        CODE HERE
    - match based on name, country, location and population
    - population/location comparison
    - 

## Sample
    - method/choice
        CODE HERE
        - known cities vs springfields
        - 20 springfields, 11 matches
    - results

applied results/conclusion